In [8]:
import generate
import symae
import os
import numpy as np
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

Num GPUs Available:  0
2.4.1


2022-06-02 11:06:50.388359: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


In [3]:
import tensorflow as tf
tfk = tf.keras
tfkl = tf.keras.layers
tfkltd= tf.keras.layers.TimeDistributed
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
print(tf.__version__)

Num GPUs Available:  0
2.4.1


In [3]:
# Ok, What's in symae?
# Downsampler, DownsamplerDense, NuisanceEncoder, NuisanceEncoder1D, NuisanceEncoderDense1D, 
# SymmetricEncoder,SymmetricEncoder1D,SymmetricEncoderDense1D,DistributeZsym,LatentCat,DroppedLatentCat, 
# Mixer, Mixer1D, MixerDense1D, Upsampler, UpsamplerDense
# They are all Neural Nets. But where is the SymAE?
#####
# SymmetricEncoder: n, ntau, nr, nt, nc = input_tensor.get_shape()
# Consider a batch of 32 video samples, 
# where each sample is a 128x128 RGB image with channels_last data format, across 10 timesteps. 
# The batch input shape is (32, 10, 128, 128, 3).
#####
# NuisanceEncoder: n, ntau, nr, nt, nc = input_tensor.get_shape()
#####
# Let's Clarifying Notations Now!!!!!

In [14]:
# Parameters related to data itself
N=1000 # N is the Cardinality of the data set X, say n_X in the paper
nt=20 # nt is the number of instances in each X_i, say n_tau in the paper (Xi[1]...Xi[nt])
d=100 # d is the dimensions of each "fundamental" data, say d=dim Xi[j] (e.g. d=28*28 for mnist)

# Parameters related to SymAE
p=8 # symmetric encoder latent dimensions, p=dim Ci (Coherent Code)
q=22 # nuisance encoder latent dimensions, q=dim Ni[j] (Dissimilar Code)

# Parameters related to NeuralNets
kernel_size=5 # How to convolute, the size of the kernel
nfilters=64 # How many filters in each convolution layer

# Parameters related to my specific algorithm of generating MRA data
ne=10 # Only use g0,g1...g9
sigma=0.1 # The intensity of the noise



In [28]:
sym_enc_input=tfk.Input(shape=(nt,d,1), dtype='float32', name='sym_enc_input')
sym_enc_output=symae.SymmetricEncoder1D(kernel_size,nfilters,[2,2,2,2],[2,2,2,2],p)(sym_enc_input)
sym_encoder=tfk.Model(sym_enc_input, sym_enc_input, name='sym_encoder')
sym_encoder.summary()

Model: "sym_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sym_enc_input (InputLayer)   [(None, 20, 100, 1)]      0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [29]:
nui_enc_input=tfk.Input(shape=(nt,d,1), dtype='float32', name='sym_enc_input')
nui_enc_output=symae.SymmetricEncoder1D(kernel_size,nfilters,[2,2,2,2],[2,2,2,2],p)(nui_enc_input)
nui_encoder=tfk.Model(nui_enc_input, nui_enc_input, name='nui_encoder')
nui_encoder.summary()

Model: "nui_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sym_enc_input (InputLayer)   [(None, 20, 100, 1)]      0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [42]:
decoder_input = tfk.Input(shape=(p+q*nt), name='latentcode')
distribute_output=symae.DistributeZsym(nt, p, q)(decoder_input)
distzsym = tfk.Model(decoder_input, distribute_output, name='distzsym')
distzsym.summary()

Model: "distzsym"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
latentcode (InputLayer)      [(None, 448)]             0         
_________________________________________________________________
distribute_zsym_2 (Distribut (None, 20, 30)            0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [58]:
mixer_input = tfk.Input(shape=(nt,p+q), name='mixer_input')
mixer_output=symae.Mixer1D(kernel_size,nfilters,10,nt)(mixer_input)
mixer = tfk.Model(mixer_input, mixer_output, name='mixer') 
#mixer_input.shape=distribute_output.shape
mixer.summary()

Model: "mixer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mixer_input (InputLayer)     [(None, 20, 30)]          0         
_________________________________________________________________
mixer1d_7 (Mixer1D)          (None, 20, 20, 1)         83199     
Total params: 83,199
Trainable params: 83,071
Non-trainable params: 128
_________________________________________________________________


TensorShape([None, 20, 30])

In [54]:
encoder_input=tfk.Input(shape=(nt,d,1), dtype='float32', name='encoder_input')
znuisance=nui_encoder(encoder_input)
zsym=sym_encoder(encoder_input)
latentcat=symae.LatentCat(0.65) #0.4 droupout
encoder=tfk.Model(encoder_input, latentcat(zsym,znuisance), name="encoder") #latentcat(zsym,znuisance)
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 20, 100, 1)] 0                                            
__________________________________________________________________________________________________
sym_encoder (Functional)        (None, 20, 100, 1)   0           encoder_input[0][0]              
__________________________________________________________________________________________________
nui_encoder (Functional)        (None, 20, 100, 1)   0           encoder_input[0][0]              
__________________________________________________________________________________________________
latent_cat_1 (LatentCat)        (None, 20, 100, 2)   0           sym_encoder[3][0]                
                                                                 nui_encoder[3][0]          

In [56]:
decoder=tfk.Model(decoder_input, mixer_output, name="decoder") 
decoder.summary()

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 20, 30), dtype=tf.float32, name='mixer_input'), name='mixer_input', description="created by layer 'mixer_input'") at layer "mixer1d_5". The following previous layers were accessed without issue: []

In [ ]:
model=tf.keras.Model(encoder_input, decoder(latentcat(zsym,znuisance)) , name='autoencoder_clone')
  model.summary()
  model.compile(optimizer='adam', loss='mse')
  model.load_weights(fn)

In [11]:
a=np.array([1.0,1])

In [12]:
type(a)

numpy.ndarray